# PL

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import math
import time
import scipy.signal
from scipy.optimize import curve_fit
import scipy.stats as stats
import pandas as pd
import os
from  scipy.stats import chi2_contingency
#from statsmodels.stats.weightstats import DescrStatsW
import glob
import csv
import fileinput
from glob import glob
import re
from scipy.signal import butter, filtfilt
import scipy.integrate as spi
from scipy.stats import chi2
from scipy.signal import find_peaks

#ej de path=r'C:\Users\Sergio\Desktop\L6y7\18-04-23'
#path='C:/Users/Nicolás Molina/Desktop/L6-7/04-05-23-14kV'
#path=r'C:\Users\Sergio\Desktop\L6y7\04-05-23-19kV'

path=r'C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\PL'

#get_ipython().run_line_magic('matplotlib', 'inline')
get_ipython().run_line_magic('matplotlib', 'qt5')

import matplotlib.ticker as ticker


## datos ya procesados

In [40]:
plt.close("all")
path=r'C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\PL'
os.chdir (path)
from scipy.signal import find_peaks
file="PL primera prueba.txt"

data = np.loadtxt(file, delimiter=' ', skiprows=1, unpack=True)

longonda=data[0]
Ps1=data[1]
Ps2=data[2]
PsNf5=data[3]
PsNf10=data[4]
PsNf15=data[5]
PsNf20=data[6]

norm=max(max(Ps1),max(Ps2),max(PsNf5),max(PsNf10),max(PsNf15),max(PsNf20)) #para normalizar

muestras=[Ps1,Ps2,PsNf5,PsNf10,PsNf15,PsNf20]
titulos=["Ps1","Ps2","Ps/Nf 5%","Ps/Nf 10%","Ps/Nf 15%","Ps/Nf 20%"]

plt.figure()
plt.title("Fotoluminiscencia de PS con diferentes cargas de Naftaleno normalizada")   #Photoluminescence of PS with different loadings of Naphtalene
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Intensidad (u.a)')
j=0

for film in muestras:
    plt.plot(longonda[20:170],film[20:170]/norm,label=titulos[j])    #divido por norm para normalizar

    j=j+1

peaks, _ = find_peaks(PsNf10,height=1000000)
peak_x = longonda[peaks]
peak_y = PsNf10[peaks]
pico=str(round(peak_x[0]))
#plt.scatter(peak_x,peak_y/norm,label=f"{pico} nm",c="black",linewidth=1,ls="-.",zorder=100,s=25)  #pico max psnf10
plt.axvline(peak_x[0],label=f"{pico} nm",c="blue",linewidth=1,ls="-.")
plt.axvline(320,ls="-",c="black",linewidth=0.8)
plt.axvline(340,ls="-",c="black",linewidth=0.8)
plt.legend()

## ple y ple cef3 


In [9]:

path=r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\cef3 solo" #path=r"C:\Users\Sergio\Desktop\tesis\Gan-AlGaN-main\adquisiciones\cef3 solo"
plt.close("all")
os.chdir (path)
import csv

file= "espectros exc y em.txt"


plple = np.loadtxt(file, delimiter=',', skiprows=4, unpack=True)

xpl=plple[0]
ypl=plple[1]

pl= xpl,ypl

normpl=max(ypl)
ypl=ypl/normpl

xple=plple[2]
yple=plple[3]

xple=xple[0:91]
yple=yple[0:91]

normple=max(yple)
yple=yple/normple



peakspl, _ = find_peaks(ypl,height=0.7)
peak_xpl = xpl[peakspl]
peak_ypl = ypl[peakspl]

peaksple, _ = find_peaks(yple,height=0.7)
peak_xple = xple[peaksple]
peak_yple = yple[peaksple]

picopl=str(round(peak_xpl[0]))
picople=str(round(peak_xple[0]))



plt.figure()
#plt.scatter(peak_x,peak_y/norm,label=f"{pico} nm",c="black",linewidth=1,ls="-.",zorder=100,s=25)  #pico max psnf10
plt.axvline(peak_xpl[0],label=f"{picopl} nm",c="blue",linewidth=1,ls="-.")
plt.axvline(peak_xple[0],label=f"{picople} nm",c="red",linewidth=1,ls="-.")



plt.title('Espectro de emisión (PL) y excitación (PLE) de las nanopartículas de $CeF_{3}$')  #XRD pattern of the synthesized $CeF_3$ nanoparticles
plt.plot(xpl,ypl,c="blue",label="PL")
plt.plot(xple,yple,c="red",linestyle="-",label="PLE")
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Intensidad (u.a)')



path=r"C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\uvvis naftaleno" #path=r"C:\Users\Sergio\Desktop\tesis\Gan-AlGaN-main\adquisiciones\uvvis naftaleno"

file= "PS1.Sample.asc"
os.chdir (path)
uvvisPS = np.loadtxt(file, delimiter=',', skiprows=86, unpack=True)

xps=uvvisPS[0]
yps=uvvisPS[1]
yps=yps/max(yps)
plt.plot(xps,yps,c="black",label="Absorbancia PS")
plt.legend()

## tiempo de vida cef3

In [17]:
plt.close("all")
path=r"C:\Users\Sergio\Desktop\tesis\Gan-AlGaN-main\adquisiciones\cef3 solo"  #'C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\cef3 solo'

os.chdir (path)

# busco la absorbancia (en uvvis) en el punto donde excito las muestras en el pl
lifetime=np.loadtxt("crudo tiempodevida.txt",delimiter=",",skiprows=8,unpack=True)
time1=lifetime[0]
int1=lifetime[0]
time2=lifetime[2]
int2=lifetime[3]


plt.figure()
plt.plot(time2,int2,label="Tiempo de vida")



ajuste=np.loadtxt("Ajuste triexp CeF3.txt",delimiter=",",skiprows=4,unpack=True)
timeajuste=ajuste[0]
intajuste=ajuste[1]

plt.plot(timeajuste,intajuste,linestyle="--",c="red",label="Ajuste triexponencial")
plt.title("Tiempo de vida de las nanopartículas de $CeF_{3}$")   #Photoluminescence of PS with different loadings of Naphtalene
plt.xlabel('Tiempo (ns)')
plt.ylabel('Intensidad (u.a)')
plt.legend()




# procesamiento completo de datos

In [12]:
plt.close("all")
path=r'C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\PL2 fail+uvvis'

os.chdir (path)

# busco la absorbancia (en uvvis) en el punto donde excito las muestras en el pl
uvvis=glob('*.csv')
baselineuv=np.loadtxt("baseline.txt",delimiter=",",skiprows=1,unpack=True)
#pl=np.loadtxt("pl2fail.txt", delimiter='	', skiprows=1, unpack=True)

def mean(numbers):
    return sum(numbers)/len(numbers)


abs=[]

print(uvvis)

uvvis=["Ps.Sample.Cycle1.Raw.csv","Ps CeF3 5 fina.Sample.Cycle1.Raw.csv","Ps CeF3 5.Sample.Cycle1.Raw.csv","Ps CeF3 10.Sample.Cycle1.Raw.csv"]

#plt.figure()
for film in uvvis:
  uv=np.loadtxt(film, delimiter=',', skiprows=1, unpack=True)
  lambda_=uv[0]
  intensidad=uv[1]-baselineuv[1]  #corrijo
  #plt.plot(lambda_[435:445],intensidad[435:445],label=film)
  abs.append(mean(intensidad[437:443]))
#plt.legend()

#ahora si empiezo con pl

pl=np.loadtxt("pl2fail2.txt", delimiter='	', skiprows=1, unpack=True)


lambdapl=pl[0]
Iblanco=pl[1]
emcorri=pl[6]
wg280=pl[7]

Is=[]


for j in [3,4,5]:
  I0=pl[j]-Iblanco
  T=10**(abs[j-2])
  fabs=1-10**(-abs[j-2])
  Ipl=I0*emcorri*100*(1-T)/(wg280*fabs)
  Ipl=I0*emcorri*100/(wg280)
  #plt.plot(lambdapl,Ipl,label=uvvis[j-2])
  Is.append(Ipl)


maxpscef5viejo=max(Is[0])
maxpscef5=max(Is[1])
maxpscef10=max(Is[2])

maxs=[maxpscef5viejo,maxpscef5,maxpscef10]

Inormalizados=Is/max(maxs)


titulos2=["Ps","Ps/CeF3 5% (old)","Ps/CeF3 5%","Ps/CeF3 10%"]

i=1
plt.figure()
plt.title("Fotoluminiscencia de PS con diferentes cargas de nanopartículas de $CeF_{3}$ normalizada")   #Photoluminescence of PS with different loadings of Naphtalene
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Intensidad (u.a)')
for j in Inormalizados:
  plt.plot(lambdapl[:137],j[:137],label=titulos2[i])
  i=i+1


PsCeF3_5old=Inormalizados[0]
peaks, _ = find_peaks(Inormalizados[0],height=0.8)
peak_x = lambdapl[peaks]
peak_y = PsCeF3_5old[peaks]
pico1=str(round(peak_x[0]))
pico2=str(round(peak_x[1]))
#plt.scatter(peak_x[0],peak_y[0],label=f"{pico1} nm",c="red",linewidth=1.1,ls="-.",zorder=100,s=25)  
#plt.scatter(peak_x[1],peak_y[1],label=f"{pico2} nm",c="red",linewidth=1.1,ls="-.",zorder=90,s=25)  
plt.axvline(peak_x[0],ls="-.",label=f"{pico1} nm",c="red",linewidth=1)
plt.axvline(peak_x[1],ls="-.",label=f"{pico2} nm",c="blue",linewidth=1)
plt.axvline(320,ls="-",c="black",linewidth=0.8)
plt.axvline(340,ls="-",c="black",linewidth=0.8)
plt.legend()






# estan raros estos datos de la de ps solo
plt.figure()
plt.title("Espectro raro de la muestra de Ps sin carga de nanopartículas")
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('Intensidad (u.a)')
plt.plot(lambdapl,pl[2],label=titulos2[0])
plt.legend()


['Ps CeF3 10.Sample.Cycle1.Raw.csv', 'Ps CeF3 5 fina.Sample.Cycle1.Raw.csv', 'Ps CeF3 5.Sample.Cycle1.Raw.csv', 'Ps.Sample.Cycle1.Raw.csv']


ValueError: could not convert string to float: 'lambda'

## uvvis cef3 14-6

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import math
import time
import scipy.signal
from scipy.optimize import curve_fit
import scipy.stats as stats
import pandas as pd
import os
from  scipy.stats import chi2_contingency
#from statsmodels.stats.weightstats import DescrStatsW
import glob

from scipy.signal import butter, filtfilt
import scipy.integrate as spi
from scipy.stats import chi2
from scipy.signal import find_peaks
from matplotlib.ticker import FuncFormatter


path=r'C:\Users\Nicolás Molina\OneDrive\Escritorio\GaNAlGaN\Gan-AlGaN\adquisiciones\uv vis cef3 14-6'  #   r"C:\Users\Sergio\Desktop\tesis\Gan-AlGaN-main\adquisiciones\uv vis cef3 14-6"

#get_ipython().run_line_magic('matplotlib', 'inline')
get_ipython().run_line_magic('matplotlib', 'qt5')

import matplotlib.ticker as ticker

os.chdir (path)
from matplotlib.ticker import FormatStrFormatter
from matplotlib.ticker import MaxNLocator
files=glob.glob('*.csv')
def format_ticks(value, pos):
    return f'{value:.2f}'  # Cambia '2' por el número de decimales que desees
# Aplicar el formato a los ticks del eje X para que solo sean enteros


plt.close("all")
titulos=["CeF3 10%","CeF3 5%","CeF3 7.3% limpia","CeF3 7.3% sucia"]
j=0
plt.figure()
for file in files:
    med = np.loadtxt(file, delimiter=',', skiprows=1, unpack=True)
    longonda=med[0]
    abs=med[1]

   
    plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))

    # Aplicar el formato a los ticks del eje Y para que solo sean enteros
    plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))

    plt.plot(longonda,abs,label=f"{titulos[j]}")
    plt.xlabel("$\lambda$ (nm)")
    plt.ylabel("Absorbancia (u.a)")
    plt.title("Absorbancias")
   
    plt.legend()   

    """
    peakshigh, _ = find_peaks(corr,height=0.025)
    peak_xhigh = t[peakshigh]
    peak_yhigh = corr[peakshigh]

    picohighprom=sum(peak_yhigh)/len(peak_yhigh)

    plt.axhline(picohighprom)
    """
    j=j+1